In [1]:
#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

import gensim
import os
import collections
import smart_open
import random
import re

In [ ]:
#add train and test files please
lee_train_file="C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/04082020_CengizHocaTalebi/Random_Articles_Full_Texts.txt"
lee_test_file="C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/04082020_CengizHocaTalebi/Random_Articles_Full_Texts.txt"

In [2]:
#Lemmatization process
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from gensim.utils import lemmatize
#from pattern.en import lemma

def lemmat(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:

        for i, line in enumerate(f):
            #tokens = gensim.utils.lemmatize(line,allowed_tags=re.compile('(NN|VB|JJ|RB)'), light=False, stopwords=frozenset({}), min_length=2, max_length=15)
            tokens=[wd.decode('utf-8').split('/')[0] for wd in lemmatize(line)]
            #tokens=[wd.decode('utf-8').split('/')[0] for wd in lemma(line)]
            if tokens_only:
                #yield str(tokens)
                yield list(tokens)
            else:
                # For training data, add tags
                yield TaggedDocument(tokens, [i])
                #yield gensim.models.doc2vec.TaggedDocument(tokens, [i])


In [ ]:
test_corpus = list(lemmat(lee_test_file, tokens_only=True))

In [ ]:
#to check how test corpus look like after lemmatization. It creates a txt file
    
with open('C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/04082020_CengizHocaTalebi/test_corpus_list_to_text.txt', 'w') as f:
    for item in test_corpus:
        #f.write("{0}\n".format(item))
        f.write("%s\n" % item)

In [ ]:
train_demo_corpus = list(lemmat(lee_train_file))

In [ ]:
#to check how train corpus look like after lemmatization. It creates a txt file
    
with open('C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/train_corpus_list_to_text.txt', 'w') as f:
    for item in train_demo_corpus:
        f.write("{0}\n".format(item))

In [7]:
#use this part for just to change the test file for lemmat option
lee_test_file="C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/04082020_CengizHocaTalebi/Test_EURADWASTE_ChemACS_ChemPC_PhilIAFOR_PsyIAFOR_PsyINPACT_CompFEDCIS.txt"
test_corpus = list(lemmat(lee_test_file, tokens_only=True))

In [ ]:
#If not preprocessing required, use this part:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
                
train_demo_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [ ]:
#check the types
print(type(test_corpus[0]))
print(type(test_corpus[:]))
print(type(train_demo_corpus[0]))
print(type(train_demo_corpus[:]))

In [ ]:
print(test_corpus[0])

In [ ]:
print(train_demo_corpus[0])

In [ ]:
#Instantiate a Doc2Vec Object
#model = gensim.models.doc2vec.Doc2Vec(vector_size=200, min_count=2, epochs=40)

#mymodel_1: for model.save
#model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40, dm=1, window=12, dm_concat=1, dbow_words=1)

#mymodel_2: for model.save
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40, dm=1, window=8)

In [ ]:
#Build a Vocabulary
model.build_vocab(train_demo_corpus)

In [ ]:
#Time to Train
%time model.train(train_demo_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
#save model
model.save("C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/mymodel_2_train_18000_plus_1random_test_cogsci_confs_plus_1random.doc2vec")

In [3]:
#load model
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec.load("C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/mymodel_2_train_18000_plus_1random_test_cogsci_confs_plus_1random.doc2vec")

In [ ]:
#Inferring a Vector
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])


In [ ]:
#Assessing Model

ranks = []
second_ranks = []
for doc_id in range(len(train_demo_corpus)):
    inferred_vector = model.infer_vector(train_demo_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [ ]:
#Let's count how each document ranks with respect to the training corpus
collections.Counter(ranks)  # Results vary between runs due to random seeding and very small corpus


In [ ]:

# Testing the Model

import pandas as pd

d = []

for doc_id in range(32400):
    reg_list_real=[]
    reg_list_ideal=[]
    
    inferred_vector = model.infer_vector(test_corpus[doc_id])
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    #print (sims)
    a, b = zip(*sims)
    
    
    reg_list_real=list(b)
    
    max= 0
    for i in reg_list_real:
        if i > max:
            max=i
      
    reg_list_ideal=[0] * 5
    
    reg_list_ideal.insert(0,100)
    
    from scipy import stats
    import numpy as np
    slope, intercept, r_value, p_value, std_err=stats.linregress(reg_list_real,reg_list_ideal)
    #print("p_value: %f" % (p_value))
    
    
    d.append({a[0]:(b[0]),a[1]:(b[1]),a[2]:(b[2]),a[3]:(b[3]),a[4]:(b[4]),a[5]:(b[5]),"P Value":p_value})

    
     
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]



pd.DataFrame(d).style.apply(highlight_max,axis=1)



In [8]:
# Testing the Model without P values

import pandas as pd

d = []

for doc_id in range(7):
    reg_list_real=[]
    reg_list_ideal=[]
    
    inferred_vector = model.infer_vector(test_corpus[doc_id])
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    #print (sims)
    a, b = zip(*sims)
    
        
    d.append({a[0]:(b[0]),a[1]:(b[1]),a[2]:(b[2]),a[3]:(b[3]),a[4]:(b[4]),a[5]:(b[5]),a[6]:(b[6])})
    #d.append({a[0]:(b[0])})
    #d.append({a[0]:(b[0]),a[1]:(b[1]),a[2]:(b[2]),a[3]:(b[3]),a[4]:(b[4]),a[5]:(b[5]),a[6]:(b[6]),a[7]:(b[7]),a[8]:(b[8]),a[9]:(b[9]),a[10]:(b[10]),a[11]:(b[11]),a[12]:(b[12]),a[13]:(b[13]),a[14]:(b[14]),a[15]:(b[15]),a[16]:(b[16]),a[17]:(b[17]),a[18]:(b[18]),a[19]:(b[19]),a[20]:(b[20]),a[21]:(b[21]),a[22]:(b[22]),a[23]:(b[23]),a[24]:(b[24]),a[25]:(b[25]),a[26]:(b[26]),a[27]:(b[27]),a[28]:(b[28]),a[29]:(b[29]),a[30]:(b[30]),a[31]:(b[31]),a[32]:(b[32]),a[33]:(b[33]),a[34]:(b[34]),a[35]:(b[35]),a[36]:(b[36]),a[37]:(b[37]),a[38]:(b[38]),a[39]:(b[39]),a[40]:(b[40]),a[41]:(b[41]),a[42]:(b[42]),a[43]:(b[43]),a[44]:(b[44]),a[45]:(b[45]),a[46]:(b[46]),a[47]:(b[47]),a[48]:(b[48]),a[49]:(b[49])})
     
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]



pd.DataFrame(d).style.apply(highlight_max,axis=1)


,0,1,2,3,4,5,6
0,0.225666,0.252306,0.123591,0.277512,0.324766,0.0752154,0.249792
1,0.204737,0.262342,0.0878656,0.459776,0.26893,0.182366,0.429132
2,0.240892,0.231798,0.173145,0.461738,0.429311,0.11021,0.69768
3,0.567484,0.438037,0.3619,0.307437,0.338041,0.34262,0.238862
4,0.253152,0.292074,0.191294,0.289149,0.103051,0.47068,0.0729354
5,0.375301,0.366288,0.218967,0.330278,0.173994,0.524088,0.13181
6,0.213411,0.253042,0.226135,0.180269,0.635897,0.115171,0.453651


In [ ]:
#Export above table as csv file

pd.DataFrame(d).style.apply(highlight_max,axis=1).to_excel("C:/Users/ENF_RP/Desktop/Journals/Doc2Vec/CogSCI/Revised_Study_After_TIK/New_Study_for_Conference_31_12_2019/mymodel_2_train_18000_plus_1random_test_cogsci_confs_plus_Euradwaste13_Phil16_Phil12_Comp16_Psy16_Chem16.xlsx")

In [ ]:
#To be sure about the doc number and text matching
for f in range(1):
    print('Test Document ({}): «{}»\n'.format(f, ' '.join(test_corpus[f][:10])))